In [1]:
import torch
import pandas as pd
import numpy as np
import gc
import traceback
import sys
from torch_operators import *

In [2]:
#base_data=pd.read_csv()读本地的基础字段csv
base_data=pd.read_csv(rf"/home/user104/Lwy_research/equity_min_strategy/test_ITwork/基础字段factor_20250221.csv")
base_data['Code']=base_data['Code'].map(lambda x:str(x).zfill(6))
code_list=sorted(base_data['Code'].dropna().unique().tolist())
time_list=sorted(base_data['time'].dropna().unique().tolist())
base_data=base_data.drop('Period',axis=1)
base_data_unstack=base_data.set_index(['Code','time']).unstack()
base_list=list(base_data.columns)[2:]

def load_base_data(base):
    return base_data_unstack[base].T.iloc[:237].values

In [ ]:
class Factor_data_test:
    def __init__(self, 
                 date,
                 base_list,
                 stock_list=[],
                 sec_list=[]):
        """_summary_

        Args:
            GlobalConfig: _description_
            date: _description_
            factor_dict: 要计算的表达式，会提前载入其中的基础字段. Defaults to {}.
            load_base_list: 需要提前载入的基础字段. Defaults to [].
            stock_list: _description_. Defaults to [].
            pre_loading : 是否需要提前按照factor_dict载入基础字段. Defaults to True.
            select_valid :规定全局股票池时是否剔除部分黑名单个股. Defaults to True.
        """
        self.base_data_freq='1m'
        self.comp_num = 240 # 基础数据空值长度，放在数据之前，用于还原min_periods,C++不需要，仅仅是因为torch不直接支持min_periods,才采取此举
        self.time_num=  237 # 最后输出的长度，不包括150000000
        self.date = date
        self._dtype ='float32'
        self.use_device = 'cpu'
        self.stock_list=stock_list
        self.base_list=base_list
            
        self.sec_list=sec_list
        # self.daily_base_data_dict={}
        # self.get_daily_base_data_dict()  #如果有数据，则预先读入

    def __getitem__(self, item):
        reindexed_data=load_base_data(item)
        comp_data = np.full((self.comp_num, len(self.stock_list)), np.nan)
        reindexed_data = np.vstack([comp_data, reindexed_data])
        return torch.from_numpy(reindexed_data).float().to(device=self.use_device)
    
    def get_base_data_name_used_in_fac_exp(self, fac_exp: str):
        name_list = [name for name in self.base_list if name in fac_exp]
        return name_list
    
    def exec_fac_exp_daily_single(self, fac_exp: str, out_device='cpu'):
            #读取单个因子
        try:
            target_data_shape = (self.time_num, len(self.stock_list))
            #valid_step=self.GlobalConfig.time_num
            fac_res = torch.empty(size=target_data_shape, device=out_device)
            base_data_name_list = self.get_base_data_name_used_in_fac_exp(fac_exp)
            local_var_dict = locals()
            for base_data_name in base_data_name_list:
                # if base_data_name in self.daily_base_data_dict.keys():
                    # comp_data=self.daily_base_data_dict[base_data_name]
                # else:
                comp_data=self[base_data_name]
                local_var_dict[rf"{base_data_name}"]=comp_data
            if ';' in fac_exp:
                exp_list = fac_exp.split(';')
                for exp in exp_list[:-1]:
                    exec(exp.strip())
                exec(rf'temp = ' + exp_list[-1].strip())
            else:
                exec(rf'temp = ' + fac_exp.strip())
            exec(rf'fac_res[:] = temp[self.comp_num:]')
            # gc.collect()
            assert fac_res.shape==target_data_shape
            return fac_res
            
        except Exception as e:
            print("An error occurred:", e)
            traceback_info = traceback.extract_tb(sys.exc_info()[2])
            filename, line_number, function_name, text = traceback_info[-1]
            print("Wrong factor:", fac_exp)
            print("Exception occurred in file:", filename)
            print("On line number:", line_number)
            print("In function:", function_name)
            print("Exception line:", text)
            del traceback_info
            gc.collect()
    
    def exec_fac_exp_daily_single_df(self, fac_exp:str):
        factor=self.exec_fac_exp_daily_single(fac_exp)
        return pd.DataFrame(factor.numpy(),index=self.sec_list[:self.time_num], columns=self.stock_list)

In [12]:
date='20250221'
fac_generator=Factor_data_test(date,base_list,code_list,time_list)
fac_x = fac_generator.exec_fac_exp_daily_single_df('y = div(add(TradeAmtSum_B,TradeAmtSum_S) , add(OrderAmtSum_B,OrderAmtSum_S)); x = div(OrderAmtSum_B , add(OrderAmtSum_B,OrderAmtSum_S)); x')
fac_y = fac_generator.exec_fac_exp_daily_single_df('y = div(add(TradeAmtSum_B,TradeAmtSum_S) , add(OrderAmtSum_B,OrderAmtSum_S)); x = div(OrderAmtSum_B , add(OrderAmtSum_B,OrderAmtSum_S)); y')
fac = fac_generator.exec_fac_exp_daily_single_df('y = div(add(TradeAmtSum_B,TradeAmtSum_S) , add(OrderAmtSum_B,OrderAmtSum_S)); x = div(OrderAmtSum_B , add(OrderAmtSum_B,OrderAmtSum_S)); ts_cross_ent(x, y, 240)')